<h1> Making an predictive model using Gradient Boosting Machine</h1>
In deze jupyter notebook file, worden de de decision tree gemaakt. 
Dit wordt gedaan om de onderzoeksvraag van mijn thesis te kunnen beantwoorden:
To what extent can support vector machine, randomforest tree, or Gradient Boosting Machine contributeto predicting the demand for the specialist youth caresegments in Amsterdam?
Ook is dit nodig voor het beantwoorden van mijn sub vraen:
•Are there neighborhood socio-demographic characteristics which are predictive of the use of youth caresegments?
•Which of the tested models has the highest f1 score in predicting the youth care segment use?


<h3>Hier onder worden eerst de benodigde librabry geimporteerd</h3>

In [ ]:
import pandas as pd
import xgboost
import time
import numpy as np
import matplotlib.pylab as plt
from matplotlib import pyplot

from sklearn import model_selection
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from numpy import mean
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from xgboost import plot_importance


Om de sub onderzoeks vraag: Which of the tested models has the highest f1-score in predicting the youth care segment use? Waarom we voor deze score hebben gekozen, kan gelezen worden onder het kopje "model eveluation".

Ook maken we een aantal variabele hier aan om de code zo gestructuurd mogelijk te houden. Waarom deze nodig zijn, valt te lezen in het kopje "model making'

In [ ]:
# define some variables
param_grid = {
        'max_depth': range(4,26,4),
        'scale_pos_weight' : [1, 25, 50, 75, 100],
        'colsample_bytree': np.arange(0.5,1.0,0.3),
}


cv_method = RepeatedStratifiedKFold(n_splits=2, 
                                    n_repeats=1, 
                                    random_state=42)


<h3> Laad de data in, die gemaakt is uit de andere jupyter notebook file</h3

In [ ]:
## load data
df = pd.read_pickle("C:\\VERTROUWELIJK\\final_dataSet.pkl")

Split the data in indepentend variable an dependent variable. 

In [ ]:
X = df.drop(['Voorziening'], axis=1)
X_encoded = pd.get_dummies(X, columns=['Geslacht'])
y = df['Voorziening'].copy()

encode string class values as integers, which is needed for Gradient Boosting

In [ ]:
label_encoder = LabelEncoder()
label_encoder = label_encoder.fit(y)
label_encoded_y = label_encoder.transform(y)

Make train and test set. Waarom dit nodig is, zie "making model" 

In [ ]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X_encoded, label_encoded_y)

<h3>Make the first gradient Boosting</h3>
and fit this to get the scores. Dit is nodig om alle onderzoeksvragen mee te bentwoorden.
Zie "model" in de thesis waarom we voro xgboost hebben gekozen en niet noor de standaard aangeleverde vanuit sklearn

In [ ]:
# fit model no training data
model = xgboost.XGBClassifier()
start_time=time.time()
# model.fit(X_train, y_train)
print(model)
scores = cross_val_score(model, X_train, y_train, cv=5, scoring='f1_micro')
score = mean(scores)
print("f1_score: %.2f%%" % (score * 100.0))

Zoals gezegd, de data is erg imbalanced. Daarom maken we een decision tree with random undersampling

In [ ]:
steps = [('under', RandomUnderSampler()), ('model', xgboost.XGBClassifier())]
pipeline = Pipeline(steps=steps)
# evaluate pipeline
cv = RepeatedStratifiedKFold(n_splits=2, n_repeats=1, random_state=42)
scores = cross_val_score(pipeline, X_encoded, label_encoded_y, scoring='f1_micro', cv=cv, n_jobs=-1, verbose = 10)
score = mean(scores)
print('F1 Score: %.3f' % score)

<h3>Prune the model</h3>
Na het vergelijken van de twee modellen, heeft het model met de normale data set de hoogste F1 score, hier na gaan we het model nog prunen. Zie Making Model in de thesis

In [ ]:
# make an gridSearch
start_time=time.time()
grid = GridSearchCV(xgboost.XGBClassifier(),
                   param_grid=param_grid,
                    scoring='f1_micro',
                   verbose=10,
                    cv= cv_method,
                   n_jobs=-1)
start_time=time.time()

grid.fit(X_train,y_train)
print(time.time()-start_time)

In [ ]:
# get the best F1 score, coming from the gridSearch.
grid.best_score_

In [ ]:
# get the best parameters of coming from the gridSearch.
grid.best_params_

Om de onderzoeks vraag: Are there neighborhood socio-demographic characteristics which are predictive of the use of youth caresegments? is onderstaande code nodig

In [ ]:
# Plot the top 10 most important features of the XGboost model
plot_importance(model, max_num_features=10)
plt.show()